In [1]:
import sagemaker
import boto3
import os

from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString
)
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline import PipelineDefinitionConfig
from sagemaker import image_uris
from steps.processor import get_processor_step
from steps.evaluation_step import get_evaluation_step

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
def get_parameters() -> dict:
    # Processing params ----------------------------
    process_instance_count_param = ParameterInteger(
        name="ProcessingInstanceCount",
        default_value=1
    )
    process_instance_type_param = ParameterString(
        name="ProcessingInstanceType",
        default_value="ml.m5.large",
    )
    #------------------------------------------------

    #------------------------------------------------

    return {
        "process_instance_count_param": process_instance_count_param,
        "process_instance_type_param": process_instance_type_param,
    }

In [11]:
def get_pipeline(
    session: sagemaker.Session,
    parameters: dict,
    constants: dict,
    sklearn_image_uri: str,
):
    pipeline_def_config = PipelineDefinitionConfig(use_custom_job_prefix=True)

    processor_step = get_processor_step(
        project=constants["project"],
        bucket_name=constants["bucket_name"],
        process_instance_count_param=parameters["process_instance_count_param"],
        process_instance_type_param=parameters["process_instance_type_param"],
        sklearn_image_uri=sklearn_image_uri,
        region=constants["region"],
    )

    # ------------------------------------------------
    evaluation_step = get_evaluation_step(
        project=constants["project"],
        bucket_name=constants["bucket_name"],
        process_instance_count_param=parameters["process_instance_count_param"],
        process_instance_type_param=parameters["process_instance_type_param"],
        evaluation_image_uri='763104351884.dkr.ecr.eu-central-1.amazonaws.com/pytorch-inference:2.3.0-gpu-py311-cu121-ubuntu20.04-ec2',
        region=constants["region"],
        
        test_metadata_prefix='sagemaker/fire-image-classification',
        best_model_prefix='models',
        test_metadata_file='test.pkl',
        best_model_file='model_resnet18.tar.gz',
        result_prefix='evaluation/mj',
        data_dir='s3://wildfires/sagemaker/fire-image-classification',
        model_package_arn='arn:aws:sagemaker:eu-central-1:567821811420:model-package/first-fire-mlflow-ee0049/1'
    )

    # ------------------------------------------------

    return Pipeline(
        name=f"{constants['project']}-pipeline-mj-1",
        parameters=[parameters[key] for key in parameters],
        pipeline_definition_config=pipeline_def_config,
        steps=[
            # processor_step,
            evaluation_step
        ],
    )

In [12]:
print(os.getcwd())

parameters = get_parameters()

constants = {
    "region": "eu-central-1",
    "project": "wildfire-project",
    "bucket_name": "wildfires",
    "sklearn_image_uri_version": "1.2-1",
}

session = sagemaker.Session(boto3.Session(region_name=constants["region"]))

sklearn_image_uri = image_uris.retrieve(
    framework="sklearn",
    region=constants["region"],
    version=constants["sklearn_image_uri_version"],
)

pipeline = get_pipeline(
    session=session,
    parameters=parameters,
    constants=constants,
    sklearn_image_uri=sklearn_image_uri,
)

pipeline.upsert(role_arn=sagemaker.get_execution_role())

execution = pipeline.start()

/home/sagemaker-user/src/pipeline


INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.


In [13]:
import time

execution_arn = execution.arn

# Monitor the execution
while True:
    status = execution.describe()['PipelineExecutionStatus']
    print(f"Pipeline execution status: {status}")
    
    if status in ['Succeeded', 'Failed', 'Stopped']:
        break
    
    time.sleep(60)  # Check every 60 seconds

if status == 'Succeeded':
    print("Pipeline execution succeeded.")
elif status == 'Failed':
    print("Pipeline execution failed.")
else:
    print("Pipeline execution stopped.")

Pipeline execution status: Executing
Pipeline execution status: Executing
Pipeline execution status: Executing
Pipeline execution status: Executing
Pipeline execution status: Executing
Pipeline execution status: Executing
Pipeline execution status: Executing
Pipeline execution status: Executing
Pipeline execution status: Succeeded
Pipeline execution succeeded.


In [9]:
s3 = boto3.client('s3')

bucket_name = 'wildfires'
s3_file_key = 'evaluation/mj/eva-result-model_resnet18'
local_file_name = 'metrics'

s3.download_file(bucket_name, s3_file_key, local_file_name)
print(f"File downloaded successfully to {local_file_name}")

File downloaded successfully to metrics


In [10]:
!cat metrics 

!rm metrics

{"metrics": {"accuracy": 0.0, "test_loss": 0.0}}